# Extract, Transform & Load on AWS platform using Python with AWS Data Wrangler

Data contains list of restaurants and their menus in USA partnered with Uber Eats. <br>

Dataset has 2 csv files- <br>
1) restaurants.csv <br>
2) restaurants-menus.csv <br>

Following task will be performed on the data: <br>

### Part A - AWS services
1) Create bucket in Amazon S3 and create 2 folders - *raw_data* & *cleaned_data*. Upload the files in raw_data. <br>
2) Create dataset in Amazon Glue. <br>
3) Create notebook instance in Amazon Sagemaker. <br>
4) Using IAM, add policies in IAM role created in Sagemaker notebook that allows to access S3 bucket, Glue database and Athena. <br>

### Part B - Using python on Jupyter platform
1) Open Jupyter platform from Sagemaker notebook. <br>
2) Install AWS Data Wrangler service. Fetch the raw data. <br>
3) Using Pandas and NumPy in python, perform data cleaning operation. <br>
4) Store the cleaned data in the bucket-folder *cleaned files* with the help of created database in Amazon Glue. <br>
5) To find out insights from restaurant database, write query using query function from AWS Athena in AWS Sagemaker. <br>


### Install AWS Data Wrangler

In [1]:
! pip install awswrangler

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 KB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 29.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.0/112.0 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 KB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 KB 30.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempt

In [3]:
!pip install zipfile36

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Import Libraries

In [4]:
import awswrangler as wr
import zipfile

### Read Data

In [14]:
r_menu = wr.s3.read_csv(path="s3://project-2-etl/raw_data/restaurant-menus.zip",compression="zip")
r_list = wr.s3.read_csv(path="s3://project-2-etl/raw_data/restaurants.csv")

In [15]:
r_list.head(1)

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703


In [16]:
r_menu.head(1)

,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD


### Delete unneccesary columns

In [17]:
r_list.drop(columns=['position','score','ratings'],inplace=True)

In [18]:
r_list.head(1)

,id,name,category,price_range,full_address,zip_code,lat,lng
0,1,PJ Fresh (224 Daniel Payne Drive),"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703


In [19]:
r_menu.drop(columns=['description'],inplace=True)

In [20]:
r_menu.head(1)

,restaurant_id,category,name,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,15.99 USD


### Renaming columns

In [21]:
r_list.rename(columns={'id':'restaurant_id','name':'restaurant_name','full_address':'address'},inplace=True)

In [22]:
r_list.head(1)

,restaurant_id,restaurant_name,category,price_range,address,zip_code,lat,lng
0,1,PJ Fresh (224 Daniel Payne Drive),"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703


In [23]:
r_menu.rename(columns={'category':'menu','name':'dish_name','price':'price_USD'},inplace=True)

In [24]:
r_menu.head(1)

,restaurant_id,menu,dish_name,price_USD
0,1,Extra Large Pizza,Extra Large Meat Lovers,15.99 USD


### Dropping Nan values

In [25]:
r_list.isna().value_counts()

restaurant_id  restaurant_name  category  price_range  address  zip_code  lat    lng  
False          False            False     False        False    False     False  False    33399
                                          True         False    False     False  False     6527
                                          False        True     True      False  False      172
                                          True         True     True      False  False      106
                                True      False        False    False     False  False       10
                                          True         False    True      False  False        9
                                                                False     False  False        4
dtype: int64

Dropping only those rows whose address and zip code is not available

In [26]:
r_list.dropna(inplace=True)

In [27]:
r_list.isna().value_counts()

restaurant_id  restaurant_name  category  price_range  address  zip_code  lat    lng  
False          False            False     False        False    False     False  False    33399
dtype: int64

In [28]:
r_menu.isna().value_counts()

restaurant_id  menu   dish_name  price_USD
False          False  False      False        3375211
dtype: int64

### Cleaning Individual Columns and Duplicate values

* Restaurant List

Check column : *restaurant_name* with *address,zip_code,lat,lng*

In [29]:
r_list.duplicated(subset=['restaurant_name','address','zip_code','lat','lng']).value_counts()

False    33382
True        17
dtype: int64

In [30]:
r_list.drop_duplicates(subset=['restaurant_name','address','zip_code','lat','lng'],keep='last',inplace=True)

In [31]:
r_list.duplicated(subset=['restaurant_name','address','zip_code','lat','lng']).value_counts()

False    33382
dtype: int64

Check column : *zip_code* : Considering 5 digit zipcode, the 1st 5 digits were considered as zipcode; for all other cases rows were deleted

In [32]:
zip = lambda x: x[:5]

list = []

for a in map(zip,r_list['zip_code']):
    try:
        var = int(a)
        list.append(var)           
    except ValueError:
        var = ''
        list.append(var)
r_list['zip_code']=list

In [33]:
r_list = r_list[r_list['zip_code']!='']
r_list['zip_code'] = r_list['zip_code'].astype(int)
r_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33377 entries, 0 to 40226
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    33377 non-null  int64  
 1   restaurant_name  33377 non-null  object 
 2   category         33377 non-null  object 
 3   price_range      33377 non-null  object 
 4   address          33377 non-null  object 
 5   zip_code         33377 non-null  int64  
 6   lat              33377 non-null  float64
 7   lng              33377 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 2.3+ MB


* Restaurant Menu

Check column : *restaurant_name*

In [34]:
r_menu.duplicated().value_counts()

False    3345541
True       29670
dtype: int64

In [35]:
r_menu.drop_duplicates(keep='last',inplace=True)

In [36]:
r_menu.duplicated().value_counts()

False    3345541
dtype: int64

Check column : *price* : Column will be converted into float type after removing USD.

In [37]:
r_menu['price_USD'] = r_menu['price_USD'].str.extract(r'(^\d*.\d*)',expand=False).astype(float)

In [49]:
r_menu.info()
r_menu.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3345541 entries, 0 to 3375210
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   restaurant_id  int64  
 1   menu           object 
 2   dish_name      object 
 3   price_usd      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 127.6+ MB


,restaurant_id,menu,dish_name,price_usd
0,1,Extra Large Pizza,Extra Large Meat Lovers,15.99
1,1,Extra Large Pizza,Extra Large Supreme,15.99
2,1,Extra Large Pizza,Extra Large Pepperoni,14.99


### Storing the cleaned files in S3 bucket having cleaned_data folder

To utilise space appropriately, *r_menu and r_list* dataframe was stored in parquet format.

In [42]:
wr.s3.to_parquet(df=r_list,path="s3://project-2-etl/cleaned_data/restaurant_list/",dataset=True,database="project-2-etl-database",table="restaurant_list")

{'paths': ['s3://project-2-etl/cleaned_data/restaurant_list/2f5217fa3adb4b36ae75245b03abb972.snappy.parquet'],
 'partitions_values': {}}

In [50]:
wr.s3.to_parquet(df=r_menu,path="s3://project-2-etl/cleaned_data/restaurants_menu/",dataset=True,database="project-2-etl-database",table="restaurants_menu")

{'paths': ['s3://project-2-etl/cleaned_data/restaurants_menu/4d2a2bb40c924cc797e836ec6b8d7991.snappy.parquet'],
 'partitions_values': {}}

### Query the required results

The query can be performed using AWS Athena also. But query is performed form this notebook by integrating Athena with the help of AWS Data Wrangler.

1) Query to find restaurants starting with *Al*

In [82]:
sql_query_1 = """
SELECT restaurant_name,price_range,address
FROM restaurant_list
WHERE restaurant_name LIKE 'Al%'
"""

In [83]:
r_list_query = wr.athena.read_sql_query(sql_query_1,database="project-2-etl-database")

In [84]:
r_list_query

,restaurant_name,price_range,address
0,Allstar Diner,$,"5704 Birmingport Rd, Sylvan Springs, AL, 35118"
1,All Original Pizzeria,$$,"215, Helena, AL, 35080"
2,Aloha Hawaiian Grill,$$,"25771 Perdido Beach Boulevard, Orange Beach, A..."
3,Albertsons Express (105 Buffalo Way),$,"105 Buffalo Way, Jackson, WY, 83001"
4,Albertsons Express (5800 Yellowstone Rd),$,"5800 Yellowstone Rd, Cheyenne, WY, 82009"
...,...,...,...
145,All About Breakfast (5610 North Interstate Hig...,$$,"5610 North Interstate Highway 35, Austin, TX, ..."
146,Aleida’s Restaurant,$,"2011 little elm trl #106 cedar park tx78613, C..."
147,All About Breakfast (200 Buttercup Creek Boule...,$$,"200 Buttercup Creek Boulevard, Cedar Park, TX,..."
148,All Star Burger (Bee Cave),$,"12921 Hill Country Blvd, Bee Cave, TX, 78738"


2) Query to find Restaurants starting with *R* with *pizza* in the menu having *price* between *10 & 40 USD* 

In [85]:
sql_query_2 = """
SELECT 
    restaurant_list.restaurant_name
    ,restaurants_menu.restaurant_id
    ,restaurants_menu.menu
    ,restaurants_menu.dish_name
    ,restaurants_menu.price_USD
FROM restaurant_list
INNER JOIN restaurants_menu
ON restaurant_list.restaurant_id=restaurants_menu.restaurant_id
WHERE (price_USD BETWEEN 10 AND 40)  AND menu LIKE '%pizza%' AND restaurant_name LIKE 'R%'
"""

In [86]:
r_comb_query = wr.athena.read_sql_query(sql_query_2,database="project-2-etl-database")

In [87]:
r_comb_query

,restaurant_name,restaurant_id,menu,dish_name,price_usd
0,Romio's Pizza &amp; Pasta (Redmond),9343,Italian specialty pizzas,Zorba Specialty Pizza - 18'' - X-Large,33.19
1,Romio's Pizza &amp; Pasta (Redmond),9343,Italian specialty pizzas,Zorba Specialty Pizza - 15'' - Large,27.85
2,Romio's Pizza &amp; Pasta (Redmond),9343,Italian specialty pizzas,Zorba Specialty Pizza - 12'' - Medium,22.49
3,Romio's Pizza &amp; Pasta (Redmond),9343,Italian specialty pizzas,Zorba Specialty Pizza - 10'' - Small,17.15
4,Romio's Pizza &amp; Pasta (Redmond),9343,Italian specialty pizzas,Zeus Specialty Pizza - 18'' - X-Large,33.19
...,...,...,...,...,...
81,Rocco's,15345,"Specialty Pizzas. 20"" or Half of a 20"" pizza",Cousin Margaret (Margherita),31.00
82,Rocco's,15345,"Specialty Pizzas. 20"" or Half of a 20"" pizza",Chilango,37.00
83,Rocco's,15345,"Specialty Pizzas. 20"" or Half of a 20"" pizza",Chicken Pesto,39.00
84,Rocco's,15345,"Specialty Pizzas. 20"" or Half of a 20"" pizza",BBQ Pizza,39.00
